In [1]:
from matplotlib import pyplot
pyplot.rcParams['figure.dpi'] = 200
pyplot.rcParams['savefig.dpi'] = 200
from sys import platform
import sys
if platform == "linux" or platform == "linux2" or platform == "darwin":
    sys.path.append("../../")
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import sympy
from src.v2.impl.conditions import StepCountCondition, PrecisionCondition, AbsolutePrecisionCondition
from src.v2.impl.methods import CoordinateDescent, GoldenRatioMethod, NewtonWolfe
from src.v2.impl.metrics import StepCount, CallCount, GradientCallCount, HessianCallCount, PrecisionCount, \
    AbsolutePrecisionCount, AbsolutePrecision, MinAbsolutePrecision
from src.v2.impl.oraculs import LambdaOracul, SymbolOracul
from src.v2.runner.debug import FULL_DEBUG
from src.v2.runner.runner import Runner, FULL_VISUALIZE, NO_VISUALIZE, FULL_ANIMATION
from src.v2.visualization.animation import Animator
from src.v2.runner.runner import TABLE
from src.v2.impl.methods import GradientDescent, ScipyMethod, Newton, NewtonBase
from IPython.display import display, HTML

display(HTML("<style>pre { white-space: pre !important; }</style>"))

def print_points(data):
    for i in data:
        print(i[0] + i[1])

In [2]:
x, y, z = sympy.symbols('x, y, z')
base = LambdaOracul(lambda x, y: (x - 10) ** 2 + (y - 5) ** 2)
base_absolute = SymbolOracul((x - 10) ** 2 + (y - 5) ** 2, ['x', 'y'])
bute = LambdaOracul(lambda x, y: (x + 2 * y - 7)**2 + (2 * x + y - 5)**2)
bute_absolute = SymbolOracul((x + 2 * y - 7)**2 + (2 * x + y - 5)**2, ['x', 'y'])
rosenbrok = LambdaOracul(lambda x, y: (1 - np.float64(x))**2 + 100 * (np.float64(y) - np.float64(x)**2)**2)
rosenbrok_absolute = SymbolOracul((1 - x)**2 + 100 * (y - x**2)**2, ['x', 'y'])
bil = LambdaOracul(lambda x, y: (1.5 - np.float64(x) + np.float64(x) * np.float64(y))**2 + (2.25 - np.float64(x) +  np.float64(x) * np.float64(y)**2)**2 + (2.625 - np.float64(x) + np.float64(x) * np.float64(y)**3)**2)
hard_non_poly = LambdaOracul(lambda x, y: np.sqrt(np.float64(x)**2+np.float64(y)**2) - np.sin(np.float64(x) + np.float64(y))**2)
macCormic = LambdaOracul(lambda x,y: np.sin(x + y) + (np.float64(x) - np.float64(y))**2 - 1.5 * np.float64(x) + 2.5 * np.float64(y) + 1)
base_non_poly = LambdaOracul(lambda x, y: np.exp(np.abs(x, dtype=np.float64), dtype=np.float64) + np.exp(np.abs(y, dtype=np.float64), dtype=np.float64))

min_bil = [3, 0.5]
min1 = np.array([10, 5])
oraculs_rosenbrok = [rosenbrok]
min2 = [1, 1]
oraculs_bute = [bute_absolute]
min3 = [1, 3]
oraculs_hard = [base_non_poly, hard_non_poly]
min4 = [0, 0]

In [3]:
metrics_base = [StepCount(), CallCount(), GradientCallCount(), HessianCallCount()]

minPrec=1e-5
defPrec=1e-7
maxPrec=1e-11

abs_metrics1 = metrics_base + [AbsolutePrecisionCount(defPrec, min1), AbsolutePrecision(min1)]
abs_metrics2 = metrics_base + [AbsolutePrecisionCount(defPrec, min2), AbsolutePrecision(min2)]
abs_metrics3 = metrics_base + [AbsolutePrecisionCount(defPrec, min3), AbsolutePrecision(min3)]
abs_metrics4 = metrics_base + [AbsolutePrecisionCount(minPrec, min4), AbsolutePrecision(min4)]
abs_metr_bil = metrics_base + [AbsolutePrecisionCount(minPrec, min_bil), AbsolutePrecision(min_bil)]

animations = [Animator()]

In [4]:
conditions = [StepCountCondition(500), PrecisionCondition(defPrec)]

modules1 = animations + abs_metrics1 + conditions
modules2 = animations + abs_metrics2 + conditions
modules3 = animations + abs_metrics3 + conditions
modules4 = animations + abs_metrics4 + [StepCountCondition(1000), PrecisionCondition(minPrec)]
modules_bil = animations + abs_metr_bil + [StepCountCondition(500), PrecisionCondition(minPrec)]

In [5]:
newton_methods = [
    NewtonBase(1),
    Newton(aprox_dec=maxPrec),
    ScipyMethod("Newton-CG")
]
point = np.array([1021.2, 3200.5])
result = Runner.run(newton_methods, [base_absolute], point, modules1, precision=defPrec, **TABLE, **FULL_VISUALIZE)


SymbolOracul
+---------------------------------------+-------------+-------------+---------------------+--------------------+---------------------------------+---------------------+
| Method name                           |   StepCount |   CallCount |   GradientCallCount |   HessianCallCount | AbsolutePrecisionCount(1e-07)   |   AbsolutePrecision |
+=======================================+=============+=============+=====================+====================+=================================+=====================+
| NewtonBase(1)                         |           1 |           0 |                   1 |                  1 | Undefined                       |      3351.68        |
+---------------------------------------+-------------+-------------+---------------------+--------------------+---------------------------------+---------------------+
| Newton(3,GoldenRatioMethod,eps=1e-11) |           1 |          57 |                   1 |                  1 | 1                           

In [6]:
point = np.array([102.1, 320.5])
result = Runner.run(newton_methods, oraculs_bute, point, modules3, precision=defPrec, **TABLE, **NO_VISUALIZE)


SymbolOracul
+---------------------------------------+-------------+-------------+---------------------+--------------------+---------------------------------+---------------------+
| Method name                           |   StepCount |   CallCount |   GradientCallCount |   HessianCallCount | AbsolutePrecisionCount(1e-07)   |   AbsolutePrecision |
+=======================================+=============+=============+=====================+====================+=================================+=====================+
| NewtonBase(1)                         |           1 |           0 |                   1 |                  1 | Undefined                       |       333.208       |
+---------------------------------------+-------------+-------------+---------------------+--------------------+---------------------------------+---------------------+
| Newton(3,GoldenRatioMethod,eps=1e-11) |           1 |          57 |                   1 |                  1 | 1                           

In [7]:
point = np.array([102.1, 320.5])
result = Runner.run(newton_methods, oraculs_rosenbrok, point, modules2, precision=defPrec, **TABLE, **FULL_VISUALIZE)


LambdaOracul
+---------------------------------------+-------------+-------------+---------------------+--------------------+---------------------------------+---------------------+
| Method name                           |   StepCount |   CallCount |   GradientCallCount |   HessianCallCount | AbsolutePrecisionCount(1e-07)   |   AbsolutePrecision |
+=======================================+=============+=============+=====================+====================+=================================+=====================+
| NewtonBase(1)                         |           1 |           0 |                   1 |                  1 | Undefined                       |       335.114       |
+---------------------------------------+-------------+-------------+---------------------+--------------------+---------------------------------+---------------------+
| Newton(3,GoldenRatioMethod,eps=1e-11) |           1 |          57 |                   1 |                  1 | Undefined                   

In [8]:
point = np.array([-100, 20.5])
methods = [
    NewtonBase(1),
    Newton(learning_rate = 10, aprox_dec=maxPrec),
    ScipyMethod("Newton-CG")
]

result = Runner.run(newton_methods, oraculs_rosenbrok, point, modules2, precision=defPrec, **TABLE, **FULL_VISUALIZE)


LambdaOracul
+---------------------------------------+-------------+-------------+---------------------+--------------------+---------------------------------+---------------------+
| Method name                           |   StepCount |   CallCount |   GradientCallCount |   HessianCallCount | AbsolutePrecisionCount(1e-07)   |   AbsolutePrecision |
+=======================================+=============+=============+=====================+====================+=================================+=====================+
| NewtonBase(1)                         |           1 |           0 |                   1 |                  1 | Undefined                       |            102.865  |
+---------------------------------------+-------------+-------------+---------------------+--------------------+---------------------------------+---------------------+
| Newton(3,GoldenRatioMethod,eps=1e-11) |           1 |          57 |                   1 |                  1 | Undefined                   

In [9]:
point = np.array([-10, 20.5])
newton_methods = [
    NewtonBase(1),
    Newton(learning_rate = 5, aprox_dec=maxPrec),
    ScipyMethod("Newton-CG")
]

result = Runner.run(newton_methods, oraculs_hard, point, modules4, precision=minPrec, **TABLE, **FULL_VISUALIZE)


LambdaOracul
+---------------------------------------+-------------+-------------+---------------------+--------------------+---------------------------------+---------------------+
| Method name                           |   StepCount |   CallCount |   GradientCallCount |   HessianCallCount | AbsolutePrecisionCount(1e-05)   |   AbsolutePrecision |
+=======================================+=============+=============+=====================+====================+=================================+=====================+
| NewtonBase(1)                         |           1 |           0 |                   1 |                  1 | Undefined                       |        22.809       |
+---------------------------------------+-------------+-------------+---------------------+--------------------+---------------------------------+---------------------+
| Newton(5,GoldenRatioMethod,eps=1e-11) |           1 |          58 |                   1 |                  1 | Undefined                   

In [10]:
print_points(result[0])

('no_show_value', array([-10. ,  20.5]), 'Method name', 'NewtonBase(1)')
('no_show_value', array([-5. , 15.5]), 'Method name', 'Newton(5,GoldenRatioMethod,eps=1e-11)')
('no_show_value', array([ 2.51191847e-06, -2.51191847e-06]), 'Method name', 'ScipyMethod(newton-cg)')


In [11]:
methods = [
    ScipyMethod("Nelder-Mead"),
    CoordinateDescent(aprox_dec=maxPrec),
    GradientDescent(learning_rate=100, aprox_dec=1e-11),
    NewtonBase(1),
    Newton(aprox_dec=1e-11, learning_rate=1),
    ScipyMethod("Newton-CG"),
    ScipyMethod("BFGS")
]

In [12]:
result = Runner.run(methods, [base, base_absolute], point, modules1, precision=defPrec, **TABLE, **NO_VISUALIZE)


LambdaOracul
+--------------------------------------------------+-------------+-------------+---------------------+--------------------+---------------------------------+---------------------+
| Method name                                      |   StepCount |   CallCount |   GradientCallCount |   HessianCallCount | AbsolutePrecisionCount(1e-07)   |   AbsolutePrecision |
+==================================================+=============+=============+=====================+====================+=================================+=====================+
| ScipyMethod(nelder-mead)                         |          39 |         145 |                   0 |                  0 | 38                              |         3.89735e-08 |
+--------------------------------------------------+-------------+-------------+---------------------+--------------------+---------------------------------+---------------------+
| CoordinateDescent(300)                           |          31 |         192 |      

In [13]:
point = np.array([102.1, 320.5])
result = Runner.run(methods, [rosenbrok, rosenbrok_absolute], point, modules2, precision=defPrec, **TABLE, **NO_VISUALIZE)


LambdaOracul
+--------------------------------------------------+-------------+-------------+---------------------+--------------------+---------------------------------+---------------------+
| Method name                                      |   StepCount |   CallCount |   GradientCallCount |   HessianCallCount | AbsolutePrecisionCount(1e-07)   |   AbsolutePrecision |
+==================================================+=============+=============+=====================+====================+=================================+=====================+
| ScipyMethod(nelder-mead)                         |         108 |         400 |                   0 |                  0 | Undefined                       |        61.3151      |
+--------------------------------------------------+-------------+-------------+---------------------+--------------------+---------------------------------+---------------------+
| CoordinateDescent(300)                           |         499 |        2039 |      

In [14]:
bad_defined_absolute = SymbolOracul(1000 * (x - 100) ** 10 + 100 * (y + 20) ** 10, ['x', 'y'])
bad_defined = LambdaOracul(lambda x, y: 1000 * (np.float64(x) - 100) ** 10 + 100 * (np.float64(y) + 20) ** 10)
min_point = np.array([100, -20])
metrics = [StepCount(), CallCount(), GradientCallCount(), HessianCallCount(),
           AbsolutePrecisionCount(0.001, min_point),
           MinAbsolutePrecision(min_point)]
conditions = [StepCountCondition(1000), PrecisionCondition(0.00001)]


modules = animations + metrics + conditions
methods2 = [
    CoordinateDescent(),
    ScipyMethod("Nelder-Mead"),
    GradientDescent(learning_rate=10, aprox_dec=0.00001),
    NewtonBase(1),
    Newton(aprox_dec=1e-14),
    ScipyMethod("Newton-CG"),
    ScipyMethod("BFGS")
]
oraculs = [bad_defined, bad_defined_absolute]
point = np.array([-100, -200])
result = Runner.run(methods2, oraculs, point, modules, precision=minPrec, **TABLE, **NO_VISUALIZE)


LambdaOracul
+-------------------------------------------------+-------------+-------------+---------------------+--------------------+---------------------------------+------------------------+
| Method name                                     |   StepCount |   CallCount |   GradientCallCount |   HessianCallCount | AbsolutePrecisionCount(0.001)   |   MinAbsolutePrecision |
+=================================================+=============+=============+=====================+====================+=================================+========================+
| CoordinateDescent(300)                          |         999 |         113 |                   0 |                  0 | Undefined                       |            1           |
+-------------------------------------------------+-------------+-------------+---------------------+--------------------+---------------------------------+------------------------+
| ScipyMethod(nelder-mead)                        |          35 |         13

In [15]:
import warnings
warnings.filterwarnings("ignore")

base_non_poly_absolute = SymbolOracul(np.e**((x+0.54719)**2) + np.e**((y+1.54719)**2), ['x', 'y'])
base_non_poly = LambdaOracul(lambda x, y:np.e**(np.float64((x+0.54719)**2)) + np.e**(np.float64((y+1.54719)**2)))
point = np.array([3, 2.5])
abs_metrics6 = metrics_base + [AbsolutePrecisionCount(minPrec, [-0.54719, -1.54719]), AbsolutePrecision([-0.54719, -1.54719])]
modules6 = animations + abs_metrics6 + conditions
methods[2] = GradientDescent(learning_rate=10, aprox_dec=1e-11)
result = Runner.run(methods, [base_non_poly, base_non_poly_absolute], point, modules6, precision=defPrec, **TABLE, **NO_VISUALIZE)


LambdaOracul
+-------------------------------------------------+-------------+-------------+---------------------+--------------------+---------------------------------+---------------------+
| Method name                                     |   StepCount |   CallCount |   GradientCallCount |   HessianCallCount | AbsolutePrecisionCount(1e-05)   |   AbsolutePrecision |
+=================================================+=============+=============+=====================+====================+=================================+=====================+
| ScipyMethod(nelder-mead)                        |          32 |         123 |                   0 |                  0 | 22                              |         2.04878e-08 |
+-------------------------------------------------+-------------+-------------+---------------------+--------------------+---------------------------------+---------------------+
| CoordinateDescent(300)                          |          19 |         144 |            

In [16]:
test = LambdaOracul(lambda x, y: np.sqrt(x**(2)+np.sin(x+y)+x**(2)+y**(2)-2 * x *y+((3)/(4))))
test_min = [-0.4442042, -0.6663063]
point = np.array([2200.1, 200.5])
methods3 = [
    CoordinateDescent(),
    ScipyMethod("Nelder-Mead"),
    GradientDescent(learning_rate=10, aprox_dec=0.00001),
    NewtonBase(1),
    Newton(),
    ScipyMethod("Newton-CG"),
    ScipyMethod("BFGS")
]
abs_metrics_test = metrics_base + [AbsolutePrecisionCount(minPrec, test_min), AbsolutePrecision(test_min)]
modules_test = animations + abs_metrics_test + [StepCountCondition(500), PrecisionCondition(minPrec)]
result = Runner.run(methods3, [test], point, modules_test, precision=defPrec, **TABLE, **FULL_VISUALIZE)


LambdaOracul
+-------------------------------------------------+-------------+-------------+---------------------+--------------------+---------------------------------+---------------------+
| Method name                                     |   StepCount |   CallCount |   GradientCallCount |   HessianCallCount | AbsolutePrecisionCount(1e-05)   |   AbsolutePrecision |
+=================================================+=============+=============+=====================+====================+=================================+=====================+
| CoordinateDescent(300)                          |         499 |         175 |                   0 |                  0 | Undefined                       |         5.88428e-05 |
+-------------------------------------------------+-------------+-------------+---------------------+--------------------+---------------------------------+---------------------+
| ScipyMethod(nelder-mead)                        |          49 |         182 |            

In [17]:
newton_methods = [
    NewtonBase(1),
    Newton(learning_rate=4, aprox_dec=defPrec),
    ScipyMethod("Newton-CG")
]

In [18]:
all_newton = newton_methods + [NewtonWolfe(c1=0.1, c2=0.9, aprox_dec=1e-7, max_iters=20)]
point = np.array([-100, -2000])
result = Runner.run(all_newton, [rosenbrok], point, modules2, precision=defPrec, **TABLE, **FULL_VISUALIZE)


LambdaOracul
+---------------------------------------+-------------+-------------+---------------------+--------------------+---------------------------------+---------------------+
| Method name                           |   StepCount |   CallCount |   GradientCallCount |   HessianCallCount | AbsolutePrecisionCount(1e-07)   |   AbsolutePrecision |
+=======================================+=============+=============+=====================+====================+=================================+=====================+
| NewtonBase(1)                         |           1 |           0 |                   1 |                  1 | Undefined                       |      2003.55        |
+---------------------------------------+-------------+-------------+---------------------+--------------------+---------------------------------+---------------------+
| Newton(4,GoldenRatioMethod,eps=1e-07) |           1 |          39 |                   1 |                  1 | Undefined                   

In [19]:
all_newton = newton_methods + [NewtonWolfe(c1=0.1, c2=0.9, aprox_dec=1e-7, max_iters=20)]
point = np.array([-100, -200])
result = Runner.run(all_newton, [bil], point, modules_bil, precision=minPrec, **TABLE, **FULL_VISUALIZE)


LambdaOracul
+---------------------------------------+-------------+-------------+---------------------+--------------------+---------------------------------+---------------------+
| Method name                           |   StepCount |   CallCount |   GradientCallCount |   HessianCallCount | AbsolutePrecisionCount(1e-05)   |   AbsolutePrecision |
+=======================================+=============+=============+=====================+====================+=================================+=====================+
| NewtonBase(1)                         |           1 |           0 |                   1 |                  1 | Undefined                       |           225.409   |
+---------------------------------------+-------------+-------------+---------------------+--------------------+---------------------------------+---------------------+
| Newton(4,GoldenRatioMethod,eps=1e-07) |           1 |          39 |                   1 |                  1 | Undefined                   

In [20]:
print_points(result[0])

('no_show_value', array([-100., -200.]), 'Method name', 'NewtonBase(1)')
('no_show_value', array([-42.85756361, -85.71317985]), 'Method name', 'Newton(4,GoldenRatioMethod,eps=1e-07)')
('no_show_value', array([ 3.56858548e-06, -8.99811040e+01]), 'Method name', 'ScipyMethod(newton-cg)')
('no_show_value', array([-4.443727e-16,  1.000000e+00]), 'Method name', 'Wolfe(0.1,0.9,eps=1e-07)')


In [21]:
point = np.array([-10, 20.5])
result = Runner.run(all_newton, [hard_non_poly], point, modules4, precision=minPrec, **TABLE, **FULL_VISUALIZE)


LambdaOracul
+---------------------------------------+-------------+-------------+---------------------+--------------------+---------------------------------+---------------------+
| Method name                           |   StepCount |   CallCount |   GradientCallCount |   HessianCallCount | AbsolutePrecisionCount(1e-05)   |   AbsolutePrecision |
+=======================================+=============+=============+=====================+====================+=================================+=====================+
| NewtonBase(1)                         |           1 |           0 |                   1 |                  1 | Undefined                       |            22.809   |
+---------------------------------------+-------------+-------------+---------------------+--------------------+---------------------------------+---------------------+
| Newton(4,GoldenRatioMethod,eps=1e-07) |           1 |          39 |                   1 |                  1 | Undefined                   

In [22]:
print_points(result[0])

('no_show_value', array([-10. ,  20.5]), 'Method name', 'NewtonBase(1)')
('no_show_value', array([5.17883281, 5.19360995]), 'Method name', 'Newton(4,GoldenRatioMethod,eps=1e-07)')
('no_show_value', array([5.3014376, 5.3014376]), 'Method name', 'ScipyMethod(newton-cg)')
('no_show_value', array([5.3014376, 5.3014376]), 'Method name', 'Wolfe(0.1,0.9,eps=1e-07)')


In [23]:
expr = (sympy.atan(x+2*y) - ((np.pi / 2) - sympy.atan(x - y)))**2
test2_abs = SymbolOracul(expr, ['x', 'y'])

In [24]:
test2 = LambdaOracul(lambda x,y: (np.arctanh(x+2*y)-(((np.pi)/(2))-np.arctanh(x-y)))**(2))
point = np.array([-4000, 0])
abs_metrics_test2 = metrics_base
modules_test2 = animations + abs_metrics_test2 + [StepCountCondition(100), PrecisionCondition(minPrec)]
result1 = Runner.run([GradientDescent(aprox_dec=1e-6), NewtonBase(), Newton(aprox_dec=1e-6)], [test2_abs], point, modules_test2, precision=minPrec, **TABLE, **FULL_VISUALIZE)


SymbolOracul
+--------------------------------------------------+-------------+-------------+---------------------+--------------------+
| Method name                                      |   StepCount |   CallCount |   GradientCallCount |   HessianCallCount |
+==================================================+=============+=============+=====================+====================+
| GradientDescent(300,GoldenRatioMethod,eps=1e-06) |          27 |        1161 |                  27 |                  0 |
+--------------------------------------------------+-------------+-------------+---------------------+--------------------+
| NewtonBase(1)                                    |           1 |           0 |                   1 |                  1 |
+--------------------------------------------------+-------------+-------------+---------------------+--------------------+
| Newton(3,GoldenRatioMethod,eps=1e-06)            |           1 |          33 |                   1 |                

In [25]:
print_points(result1[0])

('no_show_value', array([1821.24653958, 1821.24635699]), 'Method name', 'GradientDescent(300,GoldenRatioMethod,eps=1e-06)')
('no_show_value', array([-4000.,     0.]), 'Method name', 'NewtonBase(1)')
('no_show_value', array([-4.00000100e+03, -7.09909766e-21]), 'Method name', 'Newton(3,GoldenRatioMethod,eps=1e-06)')


In [26]:
def levi_function(x, y):
    FACTOR = 5
    return (np.sin(3 / FACTOR * np.pi * x)**2
            + (x / FACTOR - 1)**2 * (1 + np.sin(3 * np.pi * y / FACTOR)**2)
            + (y / FACTOR - 1)**2 * (1 + np.sin(2 * np.pi * y / FACTOR)**2))
levi = LambdaOracul(lambda x, y: levi_function(x, y))
abs_metrics5 = metrics_base + [AbsolutePrecisionCount(minPrec, [5, 5]), AbsolutePrecision([5, 5])]
modules5 = animations + abs_metrics5 + [StepCountCondition(500), PrecisionCondition(minPrec)]

In [27]:
point = np.array([102.1, 320.5])
result = Runner.run(methods, [levi], point, modules5, precision=defPrec, **TABLE, **FULL_VISUALIZE)


LambdaOracul
+-------------------------------------------------+-------------+-------------+---------------------+--------------------+---------------------------------+---------------------+
| Method name                                     |   StepCount |   CallCount |   GradientCallCount |   HessianCallCount | AbsolutePrecisionCount(1e-05)   |   AbsolutePrecision |
+=================================================+=============+=============+=====================+====================+=================================+=====================+
| ScipyMethod(nelder-mead)                        |          42 |         169 |                   0 |                  0 | Undefined                       |       307.502       |
+-------------------------------------------------+-------------+-------------+---------------------+--------------------+---------------------------------+---------------------+
| CoordinateDescent(300)                          |          27 |         154 |            